In [5]:
import pandas as pd

df = pd.read_csv('escooter_data.csv')
df = df.dropna(subset=['Start Community Area Number'])
df

,Trip ID,Start Time,End Time,Trip Distance,Trip Duration,Accuracy,Start Census Tract,End Census Tract,Start Community Area Number,End Community Area Number,Start Community Area Name,End Community Area Name,Start Centroid Latitude,Start Centroid Longitude,Start Centroid Location,End Centroid Latitude,End Centroid Longitude,End Centroid Location
10291,0dd95046-1bc8-4464-ba39-7f9e92c668b3,08/06/2019 06:00:00 PM,08/06/2019 06:00:00 PM,1004,236,152,1.703120e+10,1.703120e+10,20.0,20.0,HERMOSA,HERMOSA,41.935048,-87.737604,POINT (-87.7376041954734 41.9350481285613),41.935048,-87.737604,POINT (-87.7376041954734 41.9350481285613)
10293,6819df6a-9623-4ae0-b1d3-ec617197d229,07/08/2019 04:00:00 PM,07/08/2019 05:00:00 PM,9124,3182,10,1.703115e+10,1.703119e+10,15.0,19.0,PORTAGE PARK,BELMONT CRAGIN,41.949592,-87.761958,POINT (-87.7619580301513 41.9495915671753),41.920143,-87.760827,POINT (-87.7608269556143 41.920143229102)
10342,aa71bb53-3e7f-4964-b6e6-030800419346,08/10/2019 09:00:00 PM,08/10/2019 09:00:00 PM,502,97,152,1.703184e+10,1.703184e+10,27.0,27.0,EAST GARFIELD PARK,EAST GARFIELD PARK,41.885045,-87.713862,POINT (-87.71386248059811 41.8850448415253),41.885045,-87.713862,POINT (-87.71386248059811 41.8850448415253)
10367,5d256770-1a8a-5e20-8f78-316d1819f46b,06/25/2019 07:00:00 PM,06/25/2019 08:00:00 PM,5053,2167,0,1.703122e+10,1.703119e+10,22.0,19.0,LOGAN SQUARE,BELMONT CRAGIN,41.921126,-87.699754,POINT (-87.6997540616386 41.9211256768265),41.927819,-87.751362,POINT (-87.7513623675995 41.9278186319205)
10369,dffe5324-5f6f-48e7-9edf-d51876ba7d2f,09/19/2019 08:00:00 AM,09/19/2019 08:00:00 AM,790,111,152,1.703119e+10,1.703120e+10,19.0,20.0,BELMONT CRAGIN,HERMOSA,41.935218,-87.744389,POINT (-87.7443888733072 41.9352176386575),41.927852,-87.735628,POINT (-87.7356282729632 41.9278522690551)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710834,536f855f-2530-4c07-b93e-4de52e75cf1b,09/06/2019 07:00:00 AM,09/06/2019 07:00:00 AM,2524,500,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710835,88721836-1a6c-4ae6-b320-8e9ce1861d16,07/11/2019 11:00:00 AM,07/11/2019 11:00:00 AM,2475,480,10,NaN,NaN,20.0,19.0,HERMOSA,BELMONT CRAGIN,41.924348,-87.734740,POINT (-87.7347401601846 41.9243477570064),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710836,3f930717-06cf-499d-b390-a45282851347,09/17/2019 05:00:00 PM,09/17/2019 05:00:00 PM,2228,590,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)
710837,11d01331-0a12-607e-11d0-13310a12607e,06/21/2019 05:00:00 PM,06/21/2019 05:00:00 PM,1303,1088,10,NaN,NaN,19.0,19.0,BELMONT CRAGIN,BELMONT CRAGIN,41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187),41.927261,-87.765502,POINT (-87.7655018368979 41.9272607415187)


In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


df = df[['Start Time', 'Start Community Area Number']]
df['Start Time'] = pd.to_datetime(df['Start Time'])
df.sort_values('Start Time', inplace=True)

demand_df = df.groupby(['Start Community Area Number', pd.Grouper(key='Start Time', freq='H')]).size().reset_index(name='Demand')

pivot_df = demand_df.pivot(index='Start Time', columns='Start Community Area Number', values='Demand').fillna(0)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(pivot_df)

In [20]:
demand_df['Demand'].mean()

20.420425418594586

In [28]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

look_back = 3
X, Y = create_dataset(scaled_data, look_back)

train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(Y)]

model = Sequential()
model.add(LSTM(50, input_shape=(look_back, pivot_df.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(30, input_shape=(look_back, pivot_df.shape[1]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(pivot_df.shape[1], activation='relu'))
model.add(Dense(pivot_df.shape[1], activation='relu'))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

model.fit(X_train, Y_train, epochs=1500, batch_size=32, verbose=1)

test_loss = model.evaluate(X_test, Y_test, verbose=0)
print(f'Test Loss: {test_loss}')

Epoch 1/1500
60/60 [==============================] - 6s 8ms/step - loss: 0.0261
Epoch 2/1500
60/60 [==============================] - 1s 18ms/step - loss: 0.0212
Epoch 3/1500
60/60 [==============================] - 1s 13ms/step - loss: 0.0162
Epoch 4/1500
60/60 [==============================] - 1s 10ms/step - loss: 0.0139
Epoch 5/1500
60/60 [==============================] - 1s 17ms/step - loss: 0.0125
Epoch 6/1500
60/60 [==============================] - 1s 15ms/step - loss: 0.0117
Epoch 7/1500
60/60 [==============================] - 1s 17ms/step - loss: 0.0113
Epoch 8/1500
60/60 [==============================] - 1s 10ms/step - loss: 0.0111
Epoch 9/1500
60/60 [==============================] - 1s 13ms/step - loss: 0.0107
Epoch 10/1500
60/60 [==============================] - 1s 13ms/step - loss: 0.0106
Epoch 11/1500
60/60 [==============================] - 1s 12ms/step - loss: 0.0103
Epoch 12/1500
60/60 [==============================] - 1s 13ms/step - loss: 0.0103
Epoch 13/1500


In [24]:
from sklearn.metrics import mean_squared_error

predicted_demand = model.predict(X_test)

actual_demand = scaler.inverse_transform(Y_test)
predicted_demand = scaler.inverse_transform(predicted_demand)
rmse = np.sqrt(mean_squared_error(actual_demand, predicted_demand))
print(f'Test RMSE: {rmse}')


15/15 [==============================] - 1s 5ms/step
Test RMSE: 6.912165257920179
